### 절대모멘텀
- 주식 데이터를 로드 
- 파생변수 'STD-YM' 생성 -> Date 컬럼의 년-월 추출하여 대입
- 년-월 별 마지막 날의 데이터를 month_last_dt 데이터프레임에 대입
- 전월의 수정주가 값을 가지는 파생변수를 생성
- 전년도의 수정주가 값을 가지는 파생변수 생성
- 전월, 전년도의 수정주가를 가지고 거래내역을 생성
- 수익율 계산

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../../csv/AMZN.csv', index_col='Date')

In [ ]:
df.head()

In [ ]:
# index를 시계열 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')


In [ ]:
df['STD-YM'] = ""

In [ ]:
# STD-YM 컬럼을 생성 -> index에 있는 년-월 데이터를 추출
for i in range(len(df.index)):
    # i? -> 위치 값
    df['STD-YM'][i] = df.index[i].strftime('%Y-%m')

df.head()

In [ ]:
df.drop('STD-YM', axis=1, inplace=True)

In [ ]:
for i in df.index:
    # i? index 값
    df.loc[i, 'STD-YM'] = i.strftime('%Y-%m')

df.head()

In [ ]:
df.info()

In [ ]:
df.drop('STD-YM', axis=1, inplace=True)

In [ ]:
df['STD-YM'] = df.index.map(
    lambda x : x.strftime('%Y-%m')
)

In [ ]:
df.head()

In [ ]:
df2 = pd.read_csv('../../csv/AAPL.csv')
df2.head()

In [ ]:
from datetime import datetime

In [ ]:
# '1980-12-12' -> 시계열데이터 변환 ( 시계열 1980-12-12 ) -> 문자열 변환 ( 1980-12 )
df2['Date'].apply(
    lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m')
)

In [ ]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [ ]:
df2.info()

In [ ]:
df2['Date'].dt.strftime('%Y-%m')

In [ ]:
df.head()

In [ ]:
# STD-YM 데이터에서 다음 행의 데이터와 오늘의 데이터가 다른경우 
flag = df['STD-YM'].shift(-1) != df['STD-YM']
df.loc[flag]

In [ ]:
df.columns

In [ ]:
test_df = pd.DataFrame()
for i in range(len(df)-1):
    if df.iloc[i, 6] != df.iloc[i+1, 6]:
        data = df.iloc[[i]]
        test_df = pd.concat([test_df, data], axis=0)

test_df

In [ ]:
# 반복문 -> STD-YM 데이터에서 중복된 데이터를 제거하고 고유의 데이터들만 생성
list(
    set(
        df['STD-YM']
    )
)

In [ ]:
ym_list = df['STD-YM'].unique()

In [ ]:
ym_list[0]

In [ ]:
df.loc[df['STD-YM'] == ym_list[0]].tail(1)

In [ ]:
month_last_df = pd.DataFrame()

for i in ym_list:
    # print(i)
    flag = df['STD-YM'] == i
    data = df.loc[flag].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

month_last_df

In [67]:
# month_last_df에 파생변수 2개 생성
# 전월의 수정주가 (BF_1M)
# 전년도의 수정주가 (BF_12M)
# 결측치는 모두 0으로 대체

month_last_df['BF_1M'] = month_last_df['Adj Close'].shift(1).fillna(0)

In [70]:
month_last_df['BF_12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [71]:
month_last_df.iloc[[0, 1, 12]]

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1997-05-30,1.500000,1.510417,1.479167,1.500000,1.500000,2594400,1997-05,0.000000,0.0
1997-06-30,1.510417,1.598958,1.479167,1.541667,1.541667,2746800,1997-06,1.500000,0.0
1998-05-29,7.156250,7.416667,7.125000,7.343750,7.343750,8641200,1998-05,7.645833,1.5


## 거래 내역 생성
- 매수 조건
    - 전월의 수정주가 / 전년도의 수정주가 - 1  의 값이 0보다 크고 무한대가 아닌경우

In [74]:
import numpy as np

In [75]:
# df 데이터프레임에 trade 컬럼을 생성해서 ""대입
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산식
    momentum_index = month_last_df.loc[i, 'BF_1M'] / \
        month_last_df.loc[i, 'BF_12M'] - 1
    # print(momentum_index)
    flag = True if( (momentum_index > 0) & (momentum_index != np.inf)) else False

    if flag:
        signal = 'buy'
    print(f'날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}')
    df.loc[i, 'trade'] = signal

df['trade'].value_counts()

날짜 : 1997-05-30 00:00:00, 모멘텀 인덱스 : nan, flag : False, signal : 
날짜 : 1997-06-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-07-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-08-29 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-09-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-10-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-11-28 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1997-12-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1998-01-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1998-02-27 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1998-03-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1998-04-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1998-05-29 00:00:00, 모멘텀 인덱스 : 4.0972219999999995, flag : True, signal : buy
날짜 : 1998-06-30 00:00:00, 모멘텀 인덱스 : 3.7635124835648686, flag : True, signal : buy
날짜 : 1998-07-31 00:00:00, 모멘텀 인덱스 : 5.9391314002269775, 

C:\Users\moons\AppData\Local\Temp\ipykernel_3572\457790073.py:8: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] / \
C:\Users\moons\AppData\Local\Temp\ipykernel_3572\457790073.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] / \


trade
       5370
buy     193
Name: count, dtype: int64